# VacationPy
----

#### Note
* Instructions have been included for each segment. You do not have to follow them exactly, but they are included to help you think through the steps.

In [1]:
# Dependencies and Setup
import matplotlib.pyplot as plt
import pandas as pd
import numpy as np
import requests
import gmaps
import os
from pprint import pprint

# Import API key
from api_keys import g_key

### Store Part I results into DataFrame
* Load the csv exported in Part I to a DataFrame

In [2]:
data_file = os.path.join("output_data","cities.csv")
weather_df= pd.read_csv(data_file)
weather_df.head()


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
0,Qaanaaq,GL,77.48,-69.36,41.23,66,0,6.31,1592101871
1,Dingzhou,CN,38.51,115.00,86.00,22,0,4.47,1592101998
2,Gijang,KR,35.24,129.21,77.00,83,90,10.29,1592101998
3,Mar del Plata,AR,-38.00,-57.56,42.01,65,0,12.75,1592101876
4,Port Lincoln,AU,-34.73,135.87,61.50,72,88,23.49,1592101874


### Humidity Heatmap
* Configure gmaps.
* Use the Lat and Lng as locations and Humidity as the weight.
* Add Heatmap layer to map.

In [3]:
# Access maps with unique API key
gmaps.configure(api_key=g_key)

In [4]:
# Store latitude and longitude in locations
locations = weather_df[["Latitude", "Longitude"]]

# Store Weight
humidity_values = weather_df["Humidity"].astype(float)

# Plot Heatmap
figure_layout = {
    'width': '800px',
    'height': '600px',
    'border': '1px solid black',
    'padding': '1px',
    'margin': '0 auto 0 auto'
}
fig = gmaps.figure(zoom_level=1.4, center= (51.4934,0.0098), layout=figure_layout)
# Create heat layer
heat_layer = gmaps.heatmap_layer(locations, weights=humidity_values, 
                                 dissipating=False, max_intensity=100,
                                 point_radius=3)


# Add layer
fig.add_layer(heat_layer)

# Display figure
fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…

### Create new DataFrame fitting weather criteria
* Narrow down the cities to fit weather conditions.
* Drop any rows will null values.

In [5]:
# Ideal weather conditions
hotel_df= weather_df.loc[((weather_df["Max Temp"]>80) & (weather_df["Max Temp"]<=90)) & (weather_df["Wind Speed"]<10) \
                              & (weather_df["Humidity"]<50) & (weather_df["Cloudiness"]<5)].copy()
                                                                                                     
hotel_df

# Null values were removed in the previous analysis.

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date
1,Dingzhou,CN,38.51,115.00,86.00,22,0,4.47,1592101998
32,Pahrump,US,36.21,-115.98,84.00,15,0,1.99,1592101940
69,Riyadh,SA,24.69,46.72,80.60,26,0,4.07,1592101980
245,Korla,CN,41.76,86.15,80.78,20,0,9.31,1592102037
248,Altar,MX,30.70,-111.82,88.00,13,0,3.00,1592102037
318,Biltine,TD,14.53,20.92,80.73,44,0,5.99,1592102049
362,Umm Kaddadah,SD,13.60,26.69,87.76,27,0,8.66,1592102056
370,San Angelo,US,31.46,-100.44,87.80,22,1,8.05,1592101909
373,Safaga,EG,26.73,33.94,87.80,22,0,4.70,1592102057


### Hotel Map
* Store into variable named `hotel_df`.
* Add a "Hotel Name" column to the DataFrame.
* Set parameters to search for hotels with 5000 meters.
* Hit the Google Places API for each city's coordinates.
* Store the first Hotel result into the DataFrame.
* Plot markers on top of the heatmap.

In [6]:
hotel_df["Hotel Name"]= ""
hotel_df["Lat"]= ""
hotel_df["Lng"]= ""
hotel_df

,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Lat,Lng
1,Dingzhou,CN,38.51,115.00,86.00,22,0,4.47,1592101998,,,
32,Pahrump,US,36.21,-115.98,84.00,15,0,1.99,1592101940,,,
69,Riyadh,SA,24.69,46.72,80.60,26,0,4.07,1592101980,,,
245,Korla,CN,41.76,86.15,80.78,20,0,9.31,1592102037,,,
248,Altar,MX,30.70,-111.82,88.00,13,0,3.00,1592102037,,,
318,Biltine,TD,14.53,20.92,80.73,44,0,5.99,1592102049,,,
362,Umm Kaddadah,SD,13.60,26.69,87.76,27,0,8.66,1592102056,,,
370,San Angelo,US,31.46,-100.44,87.80,22,1,8.05,1592101909,,,
373,Safaga,EG,26.73,33.94,87.80,22,0,4.70,1592102057,,,


In [7]:
params = {
    "radius": 5000,
    "type":"lodging",
    "keyword":"hotel",
    "key": g_key
}

for index, row in hotel_df.iterrows():
    # get lat, lng from df
    lat = row["Latitude"]
    lng = row["Longitude"]
# change location each iteration while leaving original params in place
    params["location"] = f"{lat},{lng}"

    # Use the search term: "International Airport" and our lat/lng
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # make request and print url
    hotel_response = requests.get(base_url, params=params).json()
    
    try:
    
        print(f"Nice! we have found {hotel_response['results'][0]['name']} in {row['City']}")
        
        hotel_df.loc[index,'Hotel Name'] = hotel_response['results'][0]['name']
        hotel_df.loc[index,'Lat'] = hotel_response['results'][0]['geometry']['location']['lat']
        hotel_df.loc[index,'Lng'] = hotel_response['results'][0]['geometry']['location']['lng']
    except:
        print(f"No Hotel Found in {row['City']}")
        hotel_df.loc[index,'Hotel Name'] = np.NaN
        hotel_df.loc[index,'Lat']= np.NaN
        hotel_df.loc[index,'Lng'] = np.NaN
        pass
hotel_df.dropna(inplace=True)
        
    
    
hotel_df

Nice! we have found Dingzhou International Hotel in Dingzhou
Nice! we have found Holiday Inn Express & Suites Pahrump in Pahrump
Nice! we have found Four Seasons Hotel Riyadh At Kingdom Center in Riyadh
Nice! we have found The Garden Hotel Korla in Korla
Nice! we have found Hotel San Angel in Altar
No Hotel Found in Biltine
No Hotel Found in Umm Kaddadah
Nice! we have found La Quinta Inn by Wyndham and Conference Center San Angelo in San Angelo
Nice! we have found lagoonie Lodge Hotel in Safaga


,City,Country,Latitude,Longitude,Max Temp,Humidity,Cloudiness,Wind Speed,Date,Hotel Name,Lat,Lng
1,Dingzhou,CN,38.51,115.00,86.00,22,0,4.47,1592101998,Dingzhou International Hotel,38.509,115.02
32,Pahrump,US,36.21,-115.98,84.00,15,0,1.99,1592101940,Holiday Inn Express & Suites Pahrump,36.2079,-115.981
69,Riyadh,SA,24.69,46.72,80.60,26,0,4.07,1592101980,Four Seasons Hotel Riyadh At Kingdom Center,24.7115,46.6745
245,Korla,CN,41.76,86.15,80.78,20,0,9.31,1592102037,The Garden Hotel Korla,41.7595,86.1532
248,Altar,MX,30.70,-111.82,88.00,13,0,3.00,1592102037,Hotel San Angel,30.7162,-111.83
370,San Angelo,US,31.46,-100.44,87.80,22,1,8.05,1592101909,La Quinta Inn by Wyndham and Conference Center...,31.4155,-100.475
373,Safaga,EG,26.73,33.94,87.80,22,0,4.70,1592102057,lagoonie Lodge Hotel,26.7609,33.9455


In [8]:
# NOTE: Do not change any of the code in this cell

# Using the template add the hotel marks to the heatmap
info_box_template = """
<dl>
<dt>Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
</dl>
"""
# Store the DataFrame Row
# NOTE: be sure to update with your DataFrame name
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]
locations = hotel_df[["Lat", "Lng"]]

In [9]:
# Add marker layer ontop of heat map
markers= gmaps.marker_layer(locations,info_box_content= hotel_info)
fig.add_layer(markers)

# Display figure

fig

Figure(layout=FigureLayout(border='1px solid black', height='600px', margin='0 auto 0 auto', padding='1px', wi…